In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
image_folder = '/content/drive/MyDrive/Images_CNNTask'  # Update this to the path of your image folder
batch_size = 32
image_size = (224, 224)
num_classes = 2

In [4]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [5]:
train_generator = datagen.flow_from_directory(
    image_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 266 images belonging to 2 classes.


In [6]:
generator = datagen.flow_from_directory(
    image_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 266 images belonging to 2 classes.


In [7]:
val_generator = datagen.flow_from_directory(
    image_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 65 images belonging to 2 classes.


In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 4s 0us/step


In [9]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(2, activation='sigmoid')  # Assuming binary classification
])

In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
class_column = list(generator.class_indices.keys())[0]

In [12]:
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

In [19]:
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
epochs=15

In [20]:
for fold, (train_index, val_index) in enumerate(kfold.split(np.array(generator.filenames), np.array(generator.classes))):
    X_train, X_val = np.array(generator.filenames)[train_index], np.array(generator.filenames)[val_index]
    y_train, y_val = np.array(generator.classes)[train_index], np.array(generator.classes)[val_index]

    train_df = pd.DataFrame({'filename': X_train, 'class': y_train.astype(str)})
    val_df = pd.DataFrame({'filename': X_val, 'class': y_val.astype(str)})

    train_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=image_folder,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        y_col='class'
    )

    val_generator = datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=image_folder,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        y_col='class'
    )

    # Train the model for each fold
    model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator
    )

    # Make predictions on the validation set
    y_pred = model.predict(val_generator)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true = val_generator.classes

    # Calculate and store the metrics for the fold
    accuracy_scores.append(accuracy_score(y_true, y_pred_labels))
    precision_scores.append(precision_score(y_true, y_pred_labels))
    recall_scores.append(recall_score(y_true, y_pred_labels))
    f1_scores.append(f1_score(y_true, y_pred_labels))

Found 212 validated image filenames belonging to 2 classes.
Found 54 validated image filenames belonging to 2 classes.
Epoch 1/15
7/7 [==============================] - 6s 814ms/step - loss: 0.4642 - accuracy: 0.7594 - val_loss: 0.4171 - val_accuracy: 0.8704
Epoch 2/15
7/7 [==============================] - 6s 931ms/step - loss: 0.4627 - accuracy: 0.7594 - val_loss: 0.4187 - val_accuracy: 0.8148
Epoch 3/15
7/7 [==============================] - 5s 686ms/step - loss: 0.4715 - accuracy: 0.7170 - val_loss: 0.3979 - val_accuracy: 0.8148
Epoch 4/15
7/7 [==============================] - 7s 980ms/step - loss: 0.4456 - accuracy: 0.7500 - val_loss: 0.3930 - val_accuracy: 0.7593
Epoch 5/15
7/7 [==============================] - 5s 678ms/step - loss: 0.4335 - accuracy: 0.7877 - val_loss: 0.3822 - val_accuracy: 0.7963
Epoch 6/15
7/7 [==============================] - 5s 675ms/step - loss: 0.4108 - accuracy: 0.8113 - val_loss: 0.3124 - val_accuracy: 0.9259
Epoch 7/15
7/7 [=========================

In [21]:
avg_accuracy = np.mean(accuracy_scores)
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)


In [22]:
print("Average Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-score:", avg_f1)

Average Accuracy: 0.6157797939385813
Average Precision: 0.7285880414714243
Average Recall: 0.7613412228796845
Average F1-score: 0.7379531594315748
